In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from prepare import prepare_data
from model import Model

In [3]:
DATA_PATH = "./atlas-higgs-challenge-2014-v2.csv"

df, X_train, y_train, W_train, X_val, y_val, W_val, X_test, y_test, W_test = prepare_data(DATA_PATH)

In [ ]:
print("Number of Features: {0}".format(X_train.shape[1]))
print("Number of train samples: {0}".format(X_train.shape[0]))
print("Number of validation samples: {0}".format(X_val.shape[0]))
print("Number of test samples: {0}".format(X_test.shape[0]))

In [ ]:
print("Features:")
df.columns.tolist()

In [11]:
scale_pos_weight = (y_train[y_train["Label"]==1].count()/y_train[y_train["Label"]==0].count()).values[0]
evals_result = {}
params = {'max_depth': 10, 'eta': 0.1, 'gamma': 0.8,
          'subsample': 0.8, 'num_round': 1000, 'eval_metric': ['auc', 'error'],
          'lambda': 25, 'growth_policy': 'depthwise', 'silent': 0, 
          'objective': 'binary:logistic', 'scale_pos_weight': scale_pos_weight}
model = Model(params)

In [ ]:
model.fit(X_train, y_train, X_val, y_val, num_rounds=50)

In [1]:
y_pred = model.predict(X_train)

NameError: name 'model' is not defined

In [ ]:
model.score(W_train, y_pred, y_train, 0.8)

In [ ]:
model.plot_evals_result()

In [ ]:
y_pred = model.predict(X_train)

In [ ]:
model.plot_ams(W_train, y_pred, y_train)

In [25]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [ ]:
interact(model.plot_histogram, cutoff=(0.1,0.95,0.05), 
          W=fixed(W_train), X_train=fixed(X_train), y_train=fixed(y_train))

In [5]:
from ensemble import Ensemble

In [12]:
params_list = [
    {'max_depth': 10, 
     'eta': 0.1, 
     'gamma': 0.8,
     'subsample': 0.8, 
     'num_round': 500, 
     'eval_metric': ['auc', 'error'],
     'lambda': 25, 
     'growth_policy': 'depthwise', 
     'silent': 0, 
     'objective': 'binary:logistic', 
     'scale_pos_weight': scale_pos_weight},
    {'max_depth': 50, 
     'eta': 0.05, 
     'gamma': 1.8,
     'subsample': 0.6, 
     'num_round': 100, 
     'eval_metric': ['auc', 'error'],
     'lambda': 12, 
     'growth_policy': 
     'lossguide', 'silent': 0, 
     'objective': 'binary:logistic', 
     'scale_pos_weight': scale_pos_weight}
]

blender_params = {'max_depth': 10, 
                  'eta': 0.5, 
                  'gamma': 1.5,
                  'subsample': 0.9, 
                  'num_round': 500, 
                  'eval_metric': ['auc', 'error'],
                  'lambda': 4, 
                  'growth_policy': 'depthwise', 
                  'silent': 0, 
                  'objective': 'binary:logistic', 
                  'scale_pos_weight': scale_pos_weight}

In [13]:
ensemble = Ensemble(2, params_list, blender_params)
ensemble.load_ensemble(["classifier_0.model", "classifier_1.model"], "blender_classifier.model")

In [7]:
# ensemble.fit_predict(X_train, y_train, X_test)

In [8]:
# for i, clf in enumerate(ensemble.clfs):
#     clf.save_model("classifier_{0}.model".format(i))
# ensemble.blender_clf.save_model("blender_classifier.model")

In [21]:
y_pred = ensemble.predict(X_test)
np.max(y_pred)

0.00078059774

In [23]:
ensemble.score(W_test, y_pred, y_test, 1e-4)

0.799219908710952